In [1]:
import json
import pandas as pd
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation,digits
import re
# from keras.utils import to_categorical
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical
from keras.models import load_model
# from keras.layers import Input
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers import Flatten
# from keras.layers import Bidirectional
# from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
# from keras.models import Sequential
# from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
# from sklearn.model_selection import train_test_split
# %matplotlib inline
# from keras.optimizers import Adam
# import matplotlib.pyplot as plt
# from matplotlib import rcParams
# from random import shuffle
# figure size in inches
# rcParams['figure.figsize'] = 11.7,8.27
import word2vecReader as godin_embedding
from gensim.models import KeyedVectors

In [2]:
# seed = 7
# np.random.seed(seed)

In [3]:
def load_data_from_json(filename):
    with open(filename,'r') as f:
        return json.load(f)

In [5]:
master_dict = load_data_from_json('dataset/master_test.json')

In [6]:
master_dict

{'0_Cuadrilla': {'info': [{'snippets': "['files to delay application']",
    'target': 'Cuadrilla'}],
  'sentence': 'Cuadrilla files to delay application to frack in Lancashire'},
 '1001_Sainsbury': {'info': [{'snippets': "['warns of squeeze on high street retailers']",
    'target': 'Sainsbury'}],
  'sentence': 'Sainsbury chief warns of squeeze on high street retailers'},
 '1006_Barclays': {'info': [{'snippets': "['fined for anti-money-laundering failings']",
    'target': 'Barclays'}],
  'sentence': 'Barclays fined for anti-money-laundering failings'},
 '1007_Barclays': {'info': [{'snippets': "['fined for lax crime checks in']",
    'target': 'Barclays'}],
  'sentence': "UPDATE 3-Barclays fined for lax crime checks in 'deal of century'"},
 '1014_GSK': {'info': [{'snippets': "['file up to 20 new drugs for approval by 2020']",
    'target': 'GSK'}],
  'sentence': 'GSK aims to file up to 20 new drugs for approval by 2020'},
 '1031_National Grid': {'info': [{'snippets': "['Grid lines up 

In [7]:
def extract_data_from_dict(master_dict):
    sentence_id = []
    snippet = []
    for key in master_dict.keys():
        for info in master_dict[key]['info']:
            snippet.append(eval(info['snippets'])[0])
            sentence_id.append(key)
    return snippet,sentence_id

In [8]:
snippet,sentence_id = extract_data_from_dict(master_dict)

In [9]:
len(snippet),len(sentence_id)

(192, 192)

In [10]:
def remove_punctuation(s):
    list_punctuation = list(punctuation.replace('$',''))
    for i in list_punctuation:
        s = s.replace(i,'')
    return s

In [13]:
def clean_sentence(sentence):
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    #removes links
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    tokens = [remove_punctuation(w) for w in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

In [15]:
dataX = [clean_sentence(x) for x in snippet]

In [16]:
max_length  = 11

In [17]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [19]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [20]:
tokenizer = create_tokenizer(dataX)
vocab_size = len(tokenizer.word_index) + 1
trainX = encode_text(tokenizer, dataX, max_length)

In [21]:
model = load_model('models/aspect_model.h5')

In [23]:
pred = model.predict(trainX)

In [27]:
with open('models/l2_l1_map.json','r') as fin:
    L2_L1_pair = json.load(fin)

In [29]:
with open('models/label_encoding.json','r') as fin:
    enc = json.load(fin)

In [31]:
lable_encoding = enc['lables_index']

In [33]:
def get_class_from_pred(pred):
    return [lable_encoding[x.argmax()] for x in pred]

In [34]:
pred_class = get_class_from_pred(pred)

In [35]:
pred_class

['Financial',
 'Price Action',
 'Price Action',
 'Strategy',
 'Price Action',
 'Price Action',
 'M&A',
 'Price Action',
 'Dividend Policy',
 'Technical Analysis',
 'Coverage',
 'Price Action',
 'Coverage',
 'Technical Analysis',
 'Price Action',
 'Strategy',
 'Price Action',
 'Price Action',
 'Price Action',
 'IPO',
 'Price Action',
 'Price Action',
 'M&A',
 'Price Action',
 'Sales',
 'Price Action',
 'Price Action',
 'M&A',
 'Price Action',
 'Dividend Policy',
 'Appointment',
 'Technical Analysis',
 'Coverage',
 'IPO',
 'Signal',
 'Appointment',
 'Technical Analysis',
 'Sales',
 'Market',
 'M&A',
 'Dividend Policy',
 'Sales',
 'Price Action',
 'Price Action',
 'Company Communication',
 'Sales',
 'Risks',
 'Sales',
 'Price Action',
 'Dividend Policy',
 'M&A',
 'Rumors',
 'Price Action',
 'Technical Analysis',
 'M&A',
 'Price Action',
 'Price Action',
 'Price Action',
 'Rumors',
 'Price Action',
 'Sales',
 'Coverage',
 'Price Action',
 'Sales',
 'Options',
 'Volatility',
 'Price Action'

In [43]:
def combine_level2_and_level1(pred_class):
    aspect = []
    for level2 in pred_class:
        aspect.append(L2_L1_pair[level2][0]+"/"+level2)
    return aspect

In [44]:
aspect = combine_level2_and_level1(pred_class)

In [46]:
len(aspect),len(sentence_id),len(snippet)

(192, 192, 192)

In [60]:
result = {'results':[]}

In [61]:
for s_id,s_sn,asp in zip(sentence_id,snippet,aspect):
    result['results'].append({'id':s_id,'snippet':s_sn,'aspect_categories':asp})

In [62]:
result

{'results': [{'aspect_categories': 'Corporate/Financial',
   'id': '0_Cuadrilla',
   'snippet': 'files to delay application'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1001_Sainsbury',
   'snippet': 'warns of squeeze on high street retailers'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1006_Barclays',
   'snippet': 'fined for anti-money-laundering failings'},
  {'aspect_categories': 'Corporate/Strategy',
   'id': '1007_Barclays',
   'snippet': 'fined for lax crime checks in'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1014_GSK',
   'snippet': 'file up to 20 new drugs for approval by 2020'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1031_National Grid',
   'snippet': 'Grid lines up sale of'},
  {'aspect_categories': 'Corporate/M&A',
   'id': '1034_British American Tobacco',
   'snippet': 'accused of bribing senior politicians to sabotage'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1035_MillerCoors',
   'snippet': 'l

In [63]:
with open('results/aspect_result.json','w') as fout:
    json.dump(result,fout,indent=4)